In [1]:
import ee
import json
import numpy as np
import overpy
import requests
import shapely
import h3

from scripts import dl_utils
ee.Initialize()

/Users/ckruse/miniforge3/envs/m1-plastics/lib/python3.9/site-packages/descarteslabs/client/__init__.py:34: FutureWarning: Python version 3.9 is not supported yet. You may encounter unexpected errors.
  warnings.warn(msg, FutureWarning)


In [15]:
class Metadata(object):
    def __init__(self, coords):
        self.coords = coords
        self.metadata = {}
    
    def get_population(self, radius, year=2020):
        """
        Get total population within a rectangle centered on coords, and size radius
        inputs:
            - coords: [lon, lat]
            - radius: distance in kilometers
            - year: optional year between 2000 and 2020
        returns: population float, or -1 if there is an error
        """

        # Create a search box as a geojson
        bbox = dl_utils.rect_from_point([self.coords[0], self.coords[1]], radius / 111.1)
        geojson = {"type":"FeatureCollection",
            "features":
                [{"type":"Feature",
                    "properties":{},
                    "geometry":
                        bbox
                }]
        }
        geojson = json.dumps(geojson)

        # Query the worldpop API. This api is limited to 1000 calls per day without a key
        # I've applied for a key, but have not received a reply yet
        try:
            r = requests.get(f'https://api.worldpop.org/v1/services/stats?dataset=wpgppop&year={year}&geojson={geojson}&runasync=false')
            pop = r.json()['data']['total_population']
            self.metadata[f'Population - {radius} km'] = pop
        except Exception as e:
            print("failed", self.coords)
            print(e)
            self.metadata[f'Population - {radius} km'] = -1


    def get_waterways(self, plot=False):
        """
        Query OSM to get waterway nearest to point of interest
        inputs:
            - coords: [lon, lat]
            - plot: optional arg to plot waterways
        returns:
            - distance: distance to nearest waterway in meters
                        if no waterway is within 5000m, it will return -1
        """
        (lon, lat) = self.coords
        api = overpy.Overpass(max_retry_count=10, retry_timeout=30)
        result = api.query(f"""
                            (
                            way
                            (around:5000,{lat},{lon})
                            [water];
                        >;
                        way
                            (around:5000,{lat},{lon})
                            [natural=coastline];
                        >;
                        way
                            (around:5000,{lat},{lon})
                            [waterway];
                        >;);out;
                        """)
        result.get_ways()

        site_distances = []
        if plot:
            import matplotlib.pyplot as plt
            plt.figure(figsize=(6,4), dpi=150)
        for way in result.ways:
            river_coords = [[float(node.lon), float(node.lat)] for node in way.nodes]
            river = shapely.geometry.LineString(river_coords)
            point = shapely.geometry.Point(self.coords)
            site_distances.append(point.distance(river) * 111.1 * 1000)
            if plot:
                plt.plot(np.array(river_coords)[:,0], np.array(river_coords)[:,1])
        if plot:
            plt.scatter(lon, lat, c='r')
            plt.axis('equal')
            plt.show()
        if len(site_distances) > 0:
            distance = np.min(site_distances)
        else:
            print(f"No waterways found within 5 km for site at [{lon:.3f},{lat:.3f}]")
            distance = -1

        self.metadata[f'Distance to Waterway (m)'] = distance


    def sample_gee_data(self, dataset_name):
        dataset = ee.Image(dataset_name)
        response = dataset.sampleRegions(ee.Geometry.Point(self.coords), geometries=True).getInfo()
        response = response['features'][0]['properties']
        return response

    def get_landform(self):
        dataset_name = "CSP/ERGo/1_0/Global/ALOS_landforms"
        response = self.sample_gee_data(dataset_name)
        landform_id = response['constant']
        landform_descriptions = {
            11: 'Peak/ridge (warm)', 
            12:	'Peak/ridge',
            13:	'Peak/ridge (cool)',
            14:	'Mountain/divide',
            15:	'Cliff',
            21:	'Upper slope (warm)',
            22:	'Upper slope',
            23:	'Upper slope (cool)',
            24:	'Upper slope (flat)',
            31:	'Lower slope (warm)',
            32:	'Lower slope',
            33:	'Lower slope (cool)',
            34:	'Lower slope (flat)',
            41:	'Valley',
            42:	'Valley (narrow)',
        }
        self.metadata['Landform Type'] = landform_descriptions[landform_id]

    def get_soil_bulk(self):
        dataset_name = "OpenLandMap/SOL/SOL_BULKDENS-FINEEARTH_USDA-4A1H_M/v02"
        response = self.sample_gee_data(dataset_name)
        self.metadata['Fine Earth Density (kg / m^3)'] = response['b0']

    def get_clay_content(self):
        dataset_name = "OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02"
        response = self.sample_gee_data(dataset_name)
        self.metadata['Soil Clay Fraction'] = response['b0'] / 100

    def get_sand_content(self):
        dataset_name = "OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02"
        response = self.sample_gee_data(dataset_name)
        self.metadata['Soil Sand Fraction'] = response['b0'] / 100

    def get_soil_group(self):
        dataset_name = "OpenLandMap/SOL/SOL_GRTGROUP_USDA-SOILTAX-HAPLUDALFS_P/v01"
        response = self.sample_gee_data(dataset_name)
        value = response['grtgroup']
        self.metadata['Soil Great Group'] = value

    def get_elevation(self):
        dataset_name = "CGIAR/SRTM90_V4"
        response = self.sample_gee_data(dataset_name)
        value = response['elevation']
        self.metadata['Elevation'] = value

    def get_hydro_data(self):
        drainage_direction = {
            1: 'east',
            2: 'southeast',
            4: 'south',
            8: 'southwest',
            16: 'west',
            32: 'northwest',
            64: 'north',
            128: 'northeast',
            0: 'river mouth',
            -1: 'inland depression',
        }
        
        dataset_name = "MERIT/Hydro/v1_0_1"
        dataset = ee.Image(dataset_name)
        response = dataset.reduceRegions(ee.Geometry.Point(coords), ee.Reducer.mean()).getInfo()['features'][0]['properties']
        self.metadata['Drainage Direction'] = drainage_direction[response['dir']]
        self.metadata['Upstream Drainage Area (km^2)'] = response['upa']
        self.metadata['Height Above Nearest Drainage (m)'] = response['hnd']

    def get_slope(self):
        dataset_name = "CGIAR/SRTM90_V4"
        dataset = ee.Terrain.slope(ee.Image(dataset_name))
        result = dataset.reduceRegions(ee.Geometry.Point(coords), ee.Reducer.max()).getInfo()
        slope = result['features'][0]['properties']['max']
        self.metadata['Slope (degrees)'] = slope

    def generate_id(self, zoom=9):
        self.metadata['id'] = h3.geo_to_h3(4.567702965181374, 114.20615010232092, zoom)


In [16]:
coords = [114.20615010232092, 4.567702965181374]
metadata = Metadata(coords)
metadata.generate_id()
metadata.get_clay_content()
metadata.get_elevation()
metadata.get_hydro_data()
metadata.get_landform()
metadata.get_sand_content()
metadata.get_slope()
metadata.get_soil_bulk()
metadata.get_soil_group()
metadata.get_waterways()
for distance in [1, 5, 10]:
    metadata.get_population(distance)

print(json.dumps(metadata.metadata, sort_keys=False, indent=4))

{
    "id": "8968164992fffff",
    "Soil Clay Fraction": 0.32,
    "Elevation": 20,
    "Drainage Direction": "east",
    "Upstream Drainage Area (km^2)": 0.017042499035596848,
    "Height Above Nearest Drainage (m)": 2.3999998569488525,
    "Landform Type": "Lower slope (warm)",
    "Soil Sand Fraction": 0.4,
    "Slope (degrees)": 0.6202733540357432,
    "Fine Earth Density (kg / m^3)": 82,
    "Soil Great Group": 0,
    "Distance to Waterway (m)": 318.8570167898587,
    "Population - 1 km": 1055.96,
    "Population - 5 km": 16148.45,
    "Population - 10 km": 29211.36
}
